# Exercise 3
# Navigate the robot between any two points of its environment

--------------------------------------------

--------------------------------------------

### Content:
- <font size="4">**[A. Utils installation and downloads](#A)**</font></br>
>- <font size="3">[A.1. Install SOM and SARSA utils](#A1)</font></br>
>- <font size="3">[A.2. Utility function to save files into Collab Storage](#A2)</font></br>
>- <font size="3">[A.3. Utility function to save files into Collab Storage](#A3)</font></br>
>- <font size="3">[A.4. Download robot positions and SOM lattice from your Collab Storage](#A4)</font></br>


- <font size="4">**[B. Getting the environment: generate the actions table out of the SOM lattice](#B)**</font></br>
- <font size="4">**[C. Complete the SARSA implementation](#C)**</font></br>
>- <font size="3">[C.1. SARSA description](#C1)</font></br>
>- <font size="3">[C.2. Complete the SARSA class](#C2)</font></br>
>- <font size="3">[C.3. Upload SARSA_solution.py your Collab Storage](#C3)</font></br>
- <font size="4">**[D. SARSA simulation ](#D)**</font></br>
>- <font size="3">[D.1. Perform SARSA training (editable) ](#D1)</font></br>
>- <font size="3">[D.2. Save SOM training result ](#D2)</font></br>
- <font size="4">**[E. Result evaluation](#E)**</font></br>
- <font size="4">**[F. Robot navigation within the Neurorobotics Platform (NRP)](#F)**</font></br>
>- <font size="3">[F.1. Generate a test path for the NRP](#F1)</font></br>
>- <font size="3">[F.2. Save NRP test path](#F2)</font></br>
>- <font size="3">[F.3. Run experiment within the NRP](#F3)</font></br>
- <font size="4">**[G. Submission and grading ](#G)**</font></br>

--------------------------------------

--------------------------------------
## Introduction <a id='I'></a>
</br><div style="text-align: justify">**State–Action–Reward–State–Action (SARSA)** is an algorithm for learning a Markov decision process policy, used in the reinforcement learning area of machine learning. It was proposed by G. Rummery and M. Niranjan in a technical note [1] with the name "Modified Connectionist $Q$-Learning" (MCQ-L). The alternative name SARSA, proposed by Richard Sutton, was only mentioned as a footnote. </div></br>

</br><div style="text-align: justify">This name simply reflects the fact that the update of the $Q$-value essentially depends on the quintuple $(S_t, A_t, R_{t + 1}, S_{t + 1}, A_{t + 1})$ where $S_t$ is the current state of the agent, $A_t$ the selected action according to agent's policy, $R_{t + 1}$ the resulting reward, $S_{t + 1}$ the state the agent is transitioned to after action $A_t$ and $A_{t + 1}$ the newly selected action [2, Chapter 6.4], see also </div>

</br><div style="text-align: justify">A SARSA agent interacts with the environment and updates the policy based on actions taken, hence this is known as an on-policy learning algorithm. The $Q$-value for a state-action is updated by an error, adjusted by the learning rate alpha. $Q$-values represent the possible reward received in the next time step for taking action a in state s, plus the discounted future reward received from the next state-action observation.</div></br>

</br><div style="text-align: justify"> Learning the $Q$-values of the vanilla SARSA algorithm can be very long. This is why we will use an enhanced version of SARSA based on elegibility traces, see the edX lecture [3.6 Eligibility traces](https://edge.edx.org/courses/course-v1:EPFLx+RoboX+2017_T3/courseware/4a0168d1df774ebbbac66393975f6395/f12ba087286e4e72836985a33dbcf879/3?activate_block_id=block-v1%3AEPFLx%2BRoboX%2B2017_T3%2Btype%40vertical%2Bblock%40601b477a0a3a4f4b9fc42ec32ba4e9cf).</div></br>

****

</br>**References:**<div style="text-align: justify"> 
[1] "Online $Q$-Learning using Connectionist Systems" by G. Rummery and M. Niranjan (1994).</div>
</br><div style="text-align: justify">[2] "Reinforcement Learning: An Introduction" by R. Sutton and A. Barto (2017). </div>

### Getting started
You have now solved Exercise 1 and Exercise 2. Your Collab <span style="background-color: #f6d351">Storage</span> should hold two csv-files, namely the result of Exercise 1, that is, `robot_positions.csv`, and the result of  Exercise 2 , that is `lattice.csv`. Check whether these files exist in your Collab's <span style="background-color: #f6d351">Storage</span> before going any further. If any of these files is missing, resume to Section C of Exercise 2 or 3.</div>

---------------------------------------------------

---------------------------------------------------
## A. Utils installation and downloads from Collab <span style="background-color: #f6d351">Storage</span><a id='A'></a>

### A.1. Install the SARSA and SOM utils <a id='A1'></a>

In [ ]:
from IPython.display import clear_output
! pip uninstall epflx_robox_nrp_utils -y
! pip install --user --force-reinstall git+https://github.com/HBPNeurorobotics/EPFLx-RoboX-Neurorobotics-utils#egg=epflx_robox_nrp_utils
! pip install --upgrade "hbp-service-client==1.1.1"
clear_output()

### A.2 Utility functions to download files from your Collab <span style="background-color: #f6d351">Storage</span> <a id='A3'>

In [2]:
# Download a selected file from your Collab Storage
def download_from_collab_storage(filename, default):
    print('Downloading...')
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    if not filename:
        filename = default
    if not filename: 
        raise ValueError('Missing filename argument')
    from os import path
    try:
        clients.storage.download_file(path.join(collab_path, filename), './%(filename)s' % { 'filename': filename})
        print('The file \'%(filename)s\' has been successfully downloaded. It is located here: ./%(filename)s' % { 'filename': filename})
    except Exception:
        print('The file \'%(filename)s\' couldn\'t be found in your storage.' % { 'filename': filename})
        

def create_file_widget(default_filename):
    import ipywidgets as widgets
    style = {'description_width': 'initial'}
    input_file_widget = widgets.Text(
        description='Filename', 
        placeholder='%(default_filename)s (default)' % {'default_filename': default_filename},
        style=style,
        layout=widgets.Layout(width='50%')
    )
    return input_file_widget
        
def display_file_widgets(default_filenames):
    import ipywidgets as widgets
    from IPython.display import display
    input_file_widgets = []
    for default_filename in default_filenames:
        input_file_widget = create_file_widget(default_filename)
        display(input_file_widget) 
        input_file_widgets.append(input_file_widget)
    
    download_button = widgets.Button(description="Download")
    display(download_button)
    def button_callback(b):
        for fw, df in zip(input_file_widgets, default_filenames):
            download_from_collab_storage(filename=str(fw.value), default=df)
    download_button.on_click(button_callback)

### A.3 Utility function to save files into Collab <span style="background-color: #f6d351">Storage</span> <a id='A3'>

In [ ]:
# Utility function used to save files into your Collab Storage

"""
Save a file to the Collab Storage

:param filepath: relative path to the file
:param filetype: type of the file, e.g., 'text/csv' or 'text/x-python'
:param remove: remove the file from the Jupyter notebook space if True 
"""
def save_to_collab_storage(filepath, filetype, remove=False):
    print('Saving ...')
    import os
    from os import path
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    filename = path.basename(filepath)
    if os.path.isfile(filename):
        # erase old version of the file if it exists
        filepath = os.path.join(collab_path, filename)
        if clients.storage.exists(filepath): 
            clients.storage.delete(filepath)
        pydata = clients.storage.upload_file(filename, filepath, filetype)
        print("The file %(filename)s has been successfully saved to %(filepath)s" % {"filename": filename, "filepath": filepath})
        # remove from Jupyter notebook space, if requested
        if remove:
            os.remove(filename)
    else:
        print("Error: the file %(filename)s couldn't be found." % {"filename": filename})

### A.4. Download robot positions and SOM lattice from your Collab <span style="background-color: #f6d351">Storage</span> <a id='A4'></a>
Run the following cell to download the robot positions and the SOM lattice from your Collab <span style="background-color: #f6d351">Storage</span>.

In [ ]:
from IPython.display import clear_output
clear_output()
display_file_widgets(['robot_positions.csv', 'lattice.csv'])

-----------------------------------------------------

-----------------------------------------------------
## B. Getting the environment: generate the actions table out of the SOM lattice <a id='B'></a>

</br>
<div style="text-align: justify">In Exercise 2, we obtained an SOM-lattice that drastically reduced the original dataset while outlining its key toplogical features. In this exercise, we will alway refer to the SOM-lattice coordinates (integer coordinates), as opposed to the actual Cartesian coordinates of the environment (the robot arena). Our SARSA robot will move straight from one node of the SOM-lattice to one of its nearest neighbours. Since transitioning to a nearest neightboor can be prevented by obstacles, we define an the availability of each node through an actions table.</div>

Once you have executed the next cell, you will see:

- 2 images wich represent the transformation from Cartesian coordinates to SOM-lattice coordinates.
- your selected goal location `[vertical,horizontal]` in the SOM-lattice coordinates.
- the `actions` table with an array-like structure `[N,N, A(x,y)]`, where `N` is the size of the SOM-lattice. For each state `[x,y]`, `actions[x, y]` is a list of `A(x,y)` <= 4 actions (`0` - Down, `1` - Up, `2` - Right, `3` - Left).
<a id='viz'></a>

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_additional import SARSA_additional
sarsaad = SARSA_additional(
    csv_input_positions='robot_positions.csv', 
    csv_input_lattice='lattice.csv', 
    csv_output='actions.csv' # name of the output file containing all available actions
)
reward, actions = sarsaad.run_processing()

Run the next cell to save your result into your Collab <span style="background-color: #f6d351">Storage</span>.

In [ ]:
save_to_collab_storage(filepath='actions.csv', filetype='text/csv', remove=False)

-----------------------------------------------------

-----------------------------------------------------
## C. SARSA Implementation <a id='C'></a>

</br><div style="text-align: justify">Your task is to implement the SARSA algorithm with eligibility trace and to apply it to the environment, that is, the SOM-lattice of Exercise 2. Note that this environment is contained in `lattice.csv` by default. Taking `lattice.csv` as input, the cell [B.1. Generate your maze based on the SOM data](#B1) has generated an **actions table** which is now saved into `actions.csv`.</div>

</br><div style="text-align: justify">A template SARSA class is located in part [C.2. Complete the SARSA class](#C2). Once you have filled in the gaps of the implementation, you will instantiate this class to simulate the SARSA algorithm. </div>

</br><div style="text-align: justify">The **SARSA class template** consists of two parts: **1) SARSA initialization** and **2) TO DO**. The first part is used to initialize both the algorithm and its visualization . The code related to visualization, i.e., functions imported from `SARSA_additional` and their calls, should be left untouched. **<font color=red>The "TO DO" part is your assignement</font>**. </div>

</br><div style="text-align: justify"> The script in [C.2. Complete the SARSA class](#C2) breaks the SARSA algorithm into several functions. We recall the algorithm and describes these functions below. </div>

****


### C.1. SARSA and eligibility traces  <a id='C1'></a>
****

#### Notation

The symbols $R_t$ denotes the reward administered to the agent at time $t$.
The symbols $Q_{t}(S, A)$ and $e_{t}(S, A)$ denote respectively the $Q$-value and the eligibility trace of the state-action pair $(S, A)$ at time $t$.
The parameter $\eta$ is the *learning rate* of the algorithm, $\gamma$ is the *discount rate* of the return and $\lambda$ is the *trace decay parameter*.

****
#### Pseudo code <a id='pseudocode'></a>

`Initialize` $Q_0(S,A)$ arbitrarily for all state-action pair $(S, A)$  
`Repeat episode:`  
> `Initialize` $S = S_t$ for $t = 0$  
> `Initialize` $e_0(S_0, A)$ with $0$ for all $A \in \mathcal{A}(S_0)$  
> `Repeat step until` $S_t$ is a final state: 
> > `Select` $A_t \in \mathcal{A}(S_t)$ using the policy $\pi$ (e.g., $\epsilon$-greedy)  
> > `Perform` action $A_t$, `observe` $R_{t + 1}$ and $S_{t + 1}$   
> > `Update:`   
> > > `For each` state-action pair $(S, A)$ `do:` $e_t(S, A) = \lambda \gamma e_{t - 1}(S, A)$   
> > > $e_t(S_t, A_t) =  e_{t - 1}(S_t, A_t) + 1$  
> > > $\delta_t = R_{t + 1} + \gamma Q(S_{t + 1}, A_{t + 1}) - Q(S_t, A_t)$  
> > > `For each` state-action pair $(S, A)$ `do:` $Q_{t + 1}(S, A) = Q_t(S, A) + \eta \delta_t e_t(S_t, A_t)$  
> > > $t = t + 1$



#### Python implementation <a id='SARSA'></a>

The following pseudo code present the main functions to be used in the python`SARSA` class.

<font color=green>**while**</font> **`N_episodes`** are not done:<br>
> <font color='#2B8973'>*# __run_episode()__*</font><br>

> - <font color='#2B8973'>*# __init_episode()__*</font><br>
> - initialize the eligibility trace matrix **`self.e`** *[Nn x Nn x 4]*<br>
> - update **`self.epsilon`** ($\epsilon$-greedy policy with decay parameter `self.tau`)<br>
> - generate randomly an initial state. This will set the initial values of `self.state_x` and `self.state_y`.
<br><font color=red>**Important:**</font> check state availability through __`is_state_available()`__<br>
> - select an action from the current state: **`select_action()`**<br>

> <font color=green>**while**</font> <font color=green>True</font>:<br>
>> - update the agent state: __`update_state()`__<br>
>> - select an action: __`select_action()`__<br>
>> - update the eligibility trace: __`update_eligibility_trace()`__<br>
>> - update the expected reward: __`update_Q_values()`__<br>
>> - update the __`self.latency`__ parameter used for visualization
>> check __`is_episode_over()`__ and <font color='green'>__break__</font> the loop if so.<br>

****

</br><div style="text-align: justify">The script [C.2. Complete the SARSA class](#C2) has a "TO DO" section which consists of 5 subsections. The simulation will be possible when all parts are implemented correctly. We believe that the natural order of the sections eases off implementation. Let us describe the different parts of the "TO DO" section: </div>

**Initialization: [[1]](#R1)**<br>
 - **`init_parameters()`:** initialize all global parameters of the algorithm (parameters [description](#init) is to be found below); <font color=green>**return**</font> __`nothing`__.<br>

**SARSA epsiodes and steps: [[^]](#SARSA)**<br>
 - **`init_episode()`:** init the episode-specific parameters (parameters [description](#inie) is to be found below); <font color=green>**return**</font> __`nothing`__.<br>
 - **`run_episode()`:** run a single episode, i.e., execute repeatedly `run_step()` until the terminal state (the goal) is reached; <font color=green>**return**</font> __`nothing`__.<br>
 - **`run_step()`:** run a single step, i.e., execute all the updates related to a transition of the agent from one state to another one; <font color=green>**return**</font> __`nothing`__.<br>

**Updates:**<br>
 - **`update_state()`:** save the previous state of the agent and update its current [state](#upst) according to the selected action; <br> **Actions:** 0) Down = `x + 1`; 1) Up = `x - 1`; 2) Right = `y + 1`; 3) Left = `y - 1`. <font color=green>**return**</font> __`nothing`__.<br>
 - **`update_eligibility_trace()`:** update the eligibility trace matrix according to [the rules depicted above](#pseudocode); <font color=green>**return**</font> __`nothing`__.<br>
 - **`update_Q_values()`:** update the $Q$-values according to [the rules depicted above]; <font color=green>**return**</font> __`nothing`__.<br>
 
**Interaction:**<br>
 - **`select_action()`:** Select the next action according to an $\epsilon$-greedy policy: the parameter `self.epsilon` determines how often the agent will select the action with the highest $Q$-value.It does so with probability `1 - self.epsilon` and selects a random action otherwise; <font color=green>**return**</font> __`nothing`__.<br>
 - **`get_reward`:** <font color=green>**return**</font> `self.goal_reward` if the agent has reached the goal, returns `0.0` otherwise;

**Checks:**<br>
 - **`is_episode_over()`:** <font color=green>**return**</font> `True` is the episode is over (i.e., the goal is reached), `False` otherwise.<br>
 - **`is_state_available(x, y)`:** <font color=green>**return**</font> `True` if the agent can perform at least one action from the state defined by the lattice position `(x, y)`;  <font color=green>**return**</font> `False` otherwise.<br>
****

**Variables (<font color=dimgrey>uneditable</font>; <font color=blue>generated ([B](#B))</font>; constants; <font color=sienna>variables</font>):**<br>
 - <font color=dimgrey>**self.display:**</font> visualization mode (`'simulation'`, `'environment'`, `'square_maze'`, `'latency'`).<br>
 - <font color=dimgrey>**self.Nn:**</font> maze or SOM width.<br>
 - <font color=dimgrey>**self.lattice:**</font> SOM-lattice [_Nn_ x _Nn_ x 2], built using of the file `lattice.csv` of Exercise 2.<br>
 - <font color=dimgrey>**self.latency_list:**</font> list of the latencies used for visualization; initialized as an empty list.<br>
 - <font color=dimgrey>**self.episode:**</font> index of current episode; the initial value is `0`.<br>
 <br>
 - <font color=blue>**self.goal_position:**</font> the goal position with respect to SOM coordinates. You specify this position when running the cell [B. Generate your maze based on the SOM data](#B).<br>
 - <font color=blue>**self.actions:**</font> Array of list of actions as described in the cell [B.  Getting the environment: generate the actions table out of the SOM lattice](#B).<br>
 <br><a id='init'></a>
 - <font color=black>**self.N_episodes:**</font> number of episodes during learning.<br>
 - <font color=black>**self.tau:**</font> decay parameter of the $\epsilon$-greedy policy.<br>
 - <font color=black>**self.eta:**</font> learning rate.<br>
 - <font color=black>**self.gamma:**</font>discount factor - quantifies how far into the future a reward is still considered important for the current action.<br>
 - <font color=black>**self.lambda_e:**</font> the decay factor of the eligibility trace; the default is 0., which corresponds to no eligibility trace at all.<br><a id='rv'></a>
 - <font color=black>**self.goal_reward:**</font>  reward that the agent gets when it reaches to the goal.
 - <font color=sienna>**self.Q:**</font> $Q$-values matrix, also called expected reward matrix [*__Nn__* x *__Nn__* x __4__] (represents the 4 possible actions: **up, down, right, left**). <br>
<br><a id='inie'></a>
 - <font color=sienna>**self.e:**</font> eligibility trace matrix [*__Nn__* x *__Nn__* x __4__].<br>
 - <font color=sienna>**self.epsilon:**</font> probability of selecting an action randomly. The $\epsilon$- greedy policy makes sure the agent explores the maze.<br>
 - <font color=sienna>**self.latency:**</font> latency of the current episode; the initial value is `0`.<br>
 - <font color=sienna>**[self.x_start, self.y_start]:**</font> the random initial state of the agent at the begining of an episode.<br><a id='upst'></a>
 - <font color=sienna>**[self.state_x, self.state_y]:**</font> current state of the agent, i.e., $S_{t + 1}$).<br>
<br>
 - <font color=sienna>**[self.prev_state_x, self.prev_state_y]:**</font> previous state of the agent, i.e., $S_t$.<br>
 - <font color=sienna>**self.action:**</font> current selected action, i.e., $A_{t + 1}$.<br>
 - <font color=sienna>**self.prev_action:**</font> previous selected action, i.e., $A_t$.<br>

<br>
 
<a id='R1'></a> [1] Once all functions are implemented, you may need to go back to __`init_parameters()`__ and tune the SARSA parameters until you get a satisfying results. <br>

****

<font color=red>**IMPORTANT:**</font><br>

The output of the algorithm is the matrix of expected rewards, i.e. the $Q$-values. These values will be saved, by default, in the file `Q_values.csv`. You can upload this file into your Collab's <span style="background-color: #f6d351">Storage</span> by running the cell [D.2. Upload SARSA training result into Collab's storage](#D2). <br> 

</br><div style="text-align: justify"> **1)** Keep in mind that <font color=red>**the grading process will run only for a maze of size 12x12**</font>. Feel free to use other sizes for your tests.</div>
</br><div style="text-align: justify">**2)** Keep in mind that <font color=red>**the grading process has a timeout of 240 seconds</font>**.</div>
</br><div style="text-align: justify">**3)** If you want to save your solution, move to the [C.3. Upload SARSA function into Collab's storage](#C3) to upload it into your Collab <span style="background-color: #f6d351">Storage</span>. </div>
</br><div style="text-align: justify">**4)** If you feel ready to submit, jump to the **[submit](#Su)** cell.</div>

---------------------------------------

---------------------------------------

### C.2. Complete the SARSA class<a id='C2'></a>

In [ ]:
#%%writefile SARSA_solution.py 

# SARSA Class                                                                                               
import numpy
import time
from pylab import *
import matplotlib.pyplot as plt

from ipywidgets import IntProgress
from IPython import display


class SARSA:

    #############################################################################################################
    #                                 SARSA algorithm with eligibility trace                                    #
    #############################################################################################################

    def __init__(self, 
                 display='simulation', 
                 csv_input_positions='robot_positions.csv',
                 csv_input_lattice='lattice.csv',
                 csv_output_actions='actions.csv',
                 csv_output_q_values='Q_values.csv'
        ):
        from epflx_robox_nrp_utils.SARSA.SARSA_additional import SARSA_additional
        self.sarsaad = sarsaad = SARSA_additional(
            csv_input_positions=csv_input_positions, 
            csv_input_lattice=csv_input_lattice,
            csv_output_actions=csv_output_actions,
            csv_output_q_values=csv_output_q_values
        )
        self.display = display
        self.csv_file = csv_input_lattice

        # upload lattice information ('lattice.csv')
        self.lattice = self.sarsaad.upload_lattice(self.csv_file)
        self.Nn = len(self.lattice[0])

        # initialize training parameters
        self.init_parameters()

        # Upload data generated in part B.1:
        #     self.actions: available actions from each state
        #     self.goal_position: the selected goal position
        self.actions, self.goal_position = self.sarsaad.upload_environment()

        # initialize the state and action variables (S_{t + 1}= [x,y], A_{t + 1}  and  S_t = [x,y], A)
        self.state_x = None;      self.state_y = None;      self.action = None
        self.prev_state_x = None;  self.prev_state_y = None;  self.prev_action = None

        # List of latencies used to analyse the training process 
        # The latency is the number of steps needed by the robot to reach its goal in a given episode.
        self.latency_list = []


    def run_sarsa(self):
        T, self.f = self.sarsaad.sarsa_preparation(self.N_episodes,self.display)

        """   SARSA algorithm   """
        for self.episode in range(self.N_episodes):
            # run an episode and store the time taken to reach the goal
            self.latency = self.run_episode()

            # Latency visualization (every N-th episode)
            simdata = [self.Nn, self.episode, self.latency, self.N_episodes]       
            self.sarsaad.latency_visualization(simdata, self.Q, self.latency_list, self.display, self.f)          

        self.sarsaad.display_results(self.display,T,self.Q,self.goal_position,self.actions,self.csv_file)


    def run_episode(self):
        """   SARSA algorithm   """
        self.init_episode()
        while True:
            self.run_step()

            # Maze visualization (every step)
            siminfo = [self.Nn, self.episode, self.latency]
            actinfo = [self.state_x, self.state_y, self.prev_state_x, self.prev_state_y]
            Rinfo = self.goal_position
            Sinfo = [self.x_start, self.y_start]
            Qinfo = self.Q[self.state_x][self.state_y][:]
            self.sarsaad.maze_visualization(self.display, siminfo, actinfo, Rinfo, Sinfo, Qinfo)

            if self.is_episode_over(): break
        return self.latency  


    """======================================================================================================="""
    """                                                TO DO                                                  """
    """======================================================================================================="""


    ################################################
    """             Initialization               """
    ################################################

    def init_parameters(self): 
        self.N_episodes = 10000
        self.tau = self.N_episodes / 4 # time parameter used to define an epsilon(tau)-greedy policy
        self.eta = 0.3
        self.gamma = 0.8
        self.lambda_e = 0.6
        self.goal_reward = 10.0
        self.Q = numpy.zeros((self.Nn, self.Nn,4))


    ################################################
    """               SARSA stages               """
    ################################################

    def init_episode(self):
        self.e = numpy.zeros((self.Nn,self.Nn,4))
        self.epsilon = np.exp(-self.episode/(self.tau/1.0))
        self.latency = 0.0

        # pick at random an initial state for each new episode
        while True:
            self.x_start = numpy.random.randint(0, self.Nn)
            self.y_start = numpy.random.randint(0, self.Nn)
            if self.is_state_available(self.x_start, self.y_start): break

        self.state_x = self.x_start; self.state_y = self.y_start
        self.select_action()


    def run_step(self):
        self.update_state()
        self.select_action()  
        self.update_eligibility_trace()
        self.update_Q_values()
        self.latency += 1


    #########################################
    """              Updates              """
    #########################################

    def update_state(self):
        # remember the previous state of the agent
        self.prev_state_x = self.state_x
        self.prev_state_y = self.state_y

        # update the agent state according to the action
        # move down
        if self.action == 0: self.state_x += 1
        # move up
        elif self.action == 1: self.state_x -= 1
        # move right
        elif self.action == 2: self.state_y += 1
        # move left
        elif self.action == 3: self.state_y -= 1


    def update_eligibility_trace(self):
        # update the eligibility trace matrix
        self.e = self.lambda_e * self.e
        self.e[self.prev_state_x, self.prev_state_y, self.prev_action] += 1.


    def update_Q_values(self):
        # update the Q-values
        self.Q +=  self.eta * self.e * \
                      (self.get_reward() - \
                      (self.Q[self.prev_state_x,self.prev_state_y,self.prev_action] - \
                       self.gamma * self.Q[self.state_x, self.state_y, self.action]))


    #############################################
    """    Agent/Environment Interactions     """
    #############################################

    # select the next action according to an epsilon-greedy policy
    def select_action(self):
        self.prev_action = self.action
        i = None
        available_actions = self.actions[self.state_x][self.state_y]
        if numpy.random.rand() < self.epsilon:
            i = numpy.random.randint(len(available_actions))
        else:
            i = argmax(self.Q[self.state_x, self.state_y, available_actions])
        self.action = available_actions[i]


    def get_reward(self):
        # Look at next state, is it the goal?
        # move down
        if   self.prev_action == 0: self.pos_x = self.prev_state_x + 1; self.pos_y = self.prev_state_y
        # move up
        elif self.prev_action == 1: self.pos_x = self.prev_state_x - 1; self.pos_y = self.prev_state_y
        # move right
        elif self.prev_action == 2: self.pos_y = self.prev_state_y + 1; self.pos_x = self.prev_state_x
        # move left
        elif self.prev_action == 3: self.pos_y = self.prev_state_y - 1; self.pos_x = self.prev_state_x

        # return an actual reward
        if (self.pos_x == self.goal_position[0]) and (self.pos_y == self.goal_position[1]):
            return self.goal_reward
        else: return 0.0


    #########################################
    """              Checks               """
    #########################################

    def is_episode_over(self):
        # check if the agent has reached the goal OR if the latency exceeds the limit
        return (self.get_reward() == self.goal_reward) or (self.latency >= self.Nn*self.Nn)


    def is_state_available(self, x, y):
        if len(self.actions[x][y]) == 0.0:
            return False
        else: return True

****
### C.3. Upload your SARSA class implementation into the Collab <span style="background-color: #f6d351">Storage</span> <a id='C3'></a>
If you want to save the current state of your solution, follow the next steps. </br>

- Move to [C.2. Complete the SARSA class](#C2) and uncomment the first line, i.e. remove the `#` symbol before `%%writefile SARSA_solution.py` ;
- Run the cell [C.2. Complete the SARSA class](#C2); you should see `Writing SARSA_solution.py` just above; 
- Run the next cell to save this file into your Collab <span style="background-color: #f6d351">Storage</span> (check your Collab <span style="background-color: #f6d351">Storage</span> to see if the file exists).

In [ ]:
save_to_collab_storage(filepath='SARSA_solution.py', filetype='text/x-python', remove=True)

---------------------------

---------------------------
## D. SARSA simulation <a id='D'></a>
In this section, you can test your implementation of [**C.2. Complete the SARSA Class**](#C2) using different modes of visualization. You can also save the results of $Q$-values calculations into a file and upload this file to your Collab <span style="background-color: #f6d351">Storage</span>.

### D.1. Perform SARSA training <a id='D1'></a>
Executing the cell below will run [your script](#C2) and simulate your SARSA training implementation. You can visualize this process.
```python
sarsa = SARSA(display='simulation') # upload SARSA (possible values of display: 'simulation', 'environment', 'square_maze', 'latency')
sarsa.run_sarsa() # run simulation
```
`'simulation'` mode doesn't provide with any visualization;<br>
`'environment'` mode provides with visualization of robot displacement within [real environment](#viz) after every step;<br>
`'square_maze'` mode provides with visualization of robot displacement within [square maze](#viz) representing real environment after every step;<br>
`'latency'` mode provides with plotting of robot latency (number of taken steps) to come to the goal state after every episode (plot update happens every 1000 episodes).<br>
**Remark:** `'environment'` and `'square_maze'` are very consuming time mode recommended only at beginning stage to be sure that robot displaces correctly between states.

In [ ]:
sarsa = SARSA(
     display='latency', 
     csv_input_positions='robot_positions.csv',
     csv_input_lattice='lattice.csv',
     csv_output_actions='actions.csv',
     csv_output_q_values='Q_values.csv'
)
sarsa.run_sarsa()

### D.2. Save the SARSA training result <a id='D2'></a>
The result of the SARSA training is a *navigation heatmap* based on the $Q$-values. This is the table displayed after the command `sarsa.run_sarsa()` has been executed. The $Q$-values have been saved into the file `Q_values.csv` which is now contained in this Jupyter notebook. The next command will upload the `Q_values.csv` to your Collab <span style="background-color: #f6d351">Storage</span>. 
Run the cell below and check your Collab <span style="background-color: #f6d351">Storage</span> to see if the `Q_values.csv` exists.

In [ ]:
save_to_collab_storage(filepath='Q_values.csv', filetype='text/csv', remove=False)

-------------------------------

-------------------------------
## E. SARSA Evaluation <a id='E'></a>

The goal of the SARSA training is to obtain a navigation map based on $Q$-values. Such a map should provide the shortest path between any position of the robot environment and any goal position. The following command collects information about robot movements using this navigation map. By running this comand, you will get the statistics of all robot paths to the goal. You can visualize this process and check how the robot performs.

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_evaluation import SARSA_evaluation
sarsaev = SARSA_evaluation()
# Run evaluation with specific mode
from IPython.display import clear_output
try: display = input("To visualize an evaluation prosess, please, input '1': ")
except: display = 0;
clear_output()

# Analysis
best, good, over, never = sarsaev.run_evaluation(display=display, csv_input_lattice='lattice.csv')
# Statistics
print 'Agent came to the goal in', best+good, 'of', best+good+never,'cases.'
print 'Agent came to the goal by the shortest way in', best,'cases.'
print 'Agent came to the goal by non-shortest way in', good,'cases using', over, 'steps over.'
print 'Agent did not come to the goal in', never, 'cases.'

-----------------------------------------------

-----------------------------------------------
## F. Robot navigation within the Neurorobotics Platform (NRP) <a id='F'></a>
The goal of this last section is to reuse the SARSA training results in a robotics simulation.

### F.1. Generate waypoints for the NRP <a id='F1'></a>

The following cell will create the file `waypoints.csv` and fill it with waypoints joining a selected start position and the choosen goal position of the SARSA algorithm.

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_evaluation import SARSA_evaluation
sarsaev = SARSA_evaluation()
sarsaev.generate_waypoints(csv_input_lattice='lattice.csv', csv_output_waypoints='waypoints.csv')

### F.2. Save the waypoints your Collab <span style="background-color: #f6d351">Storage</span> <a id='F2'></a>
We have just generated a path for the robot navigation within the NRP. 
This path is now written into the file `waypoints.csv` contained in this Jupyter notebook space. It consists of a sequence if [x,y] coordinates the robot will go through. The following command uploads the `waypoints.csv` into your Collab <span style="background-color: #f6d351">Storage</span>. 

Run the cell below and check your Collab <span style="background-color: #f6d351">Storage</span> to see if the `waypoints.csv` exists.

In [ ]:
save_to_collab_storage(filepath='waypoints.csv', filetype='text/csv', remove=False)

### F.3. Run Exercise 3's experiment within the NRP <a id='F3'></a>

Here we use the results obtained with the SOM and SARSA algorithms. Thanks to these results, we will navigate a simulated robot within a virtual environment of the [Neurorobotics platform](http://148.187.97.48/#/esv-private?dev). To launch the relevant simulation, please, follow the instructions below.

- Download the files`lattice_data.csv` and `waypoints.csv` from your Collab <span style="background-color: #f6d351">Storage</span>.
- Join the [Neurorobotics platform](http://148.187.97.48/#/esv-private?dev) and select the ***Templates*** tab.
- Select *Exercise_3* in the list of Templates (use the search filter) and press the ***Clone*** button.
- Select the ***Experiment files*** tab and open the *Exercise_3* folder.
- Upload the files `lattice.csv` and `waypoints.csv` into the Experiment files storage.
- Move back to the ***My experiments*** tab and launch ***"Exercise_3: Navigate ..."***.
- Run the simulation and check if the robot navigates through the specified waypoints.

## G. Submission and grading
Make sure you have saved your SARSA implementation to your Collab <span style="background-color: #f6d351">Storage</span>, i.e., check the steps of [C.3. Upload your SARSA class implementation into the Collab](#C3). 

Then run the next cell to submit your SARSA implementation. Each submission is automatically tested and evaluated. Only the last submission will be taken into account for your grade. <font color=red>**Keep in mind that the grading process has a timeout of 240 seconds</font>**.</div>

In [ ]:
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
print("Retrieving your HBP OIDC token ...")
token = str(oauth.get_token())
print("token retrieved!")
submission_info = {
    'subheader': 'Exercise 3',
    'filepath': 'SARSA_solution.py',
    'token': token,
    'collab_path': collab_path,
    'clients_storage': clients.storage
}
from epflx_robox_nrp_utils.submission_manager.submission_widget import display_submission_widget
display_submission_widget(submission_info)